In [1]:
%matplotlib nbagg
%load_ext autoreload
%autoreload 2
import os
import sys
import pickle

import nltk
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

chatbot_path = "/home/bi0max/projects/tutorials/chatbot"
if chatbot_path not in sys.path:
    sys.path.append(chatbot_path)

from chatbot.config import *
from chatbot.embed_data import *
from chatbot import embed_data
from chatbot.train import *

from keras.layers import Embedding, LSTM, Input, Dense
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
print("Opening files...")
path = os.path.join(DATA_DIR, "tokenized_from.pickle")
tokenized_from = pickle.load(open(path, "rb"))
path = os.path.join(DATA_DIR, "tokenized_to.pickle")
tokenized_to = pickle.load(open(path, "rb"))

# read matrix, index
embedding_matrix = pickle.load(open(PATHS["embedding_matrix"], "rb"))
word2index = pickle.load(open(PATHS["word2index"], "rb"))

Opening files...


In [16]:
# eliminate empty sentences
def argempty(tokenized_from, tokenized_to):
    index_zeros = np.zeros(len(tokenized_from))
    for i, sentence_f, sentence_t in zip(range(len(tokenized_from)), tokenized_from, tokenized_to):
        if not sentence_f or not sentence_t:
            index_zeros[i] = 1
    return index_zeros.astype(bool)
index_zeros = argempty(tokenized_from, tokenized_to)

In [14]:
array_from = np.array(tokenized_from)
array_to = np.array(tokenized_to)

In [17]:
array_from = array_from[~index_zeros]
array_to = array_to[~index_zeros]

In [18]:
array_from.size

3767234

In [20]:
vocab = list(word2index.keys())

def argunks(tokenized, vocab, threshold=0.25):
    index_uneligible = np.zeros(len(tokenized))
    for i in range(tokenized.size):
        sentence = tokenized[i]
        n_unks = 0
        n = len(sentence)
        for word in sentence:
            if word not in vocab:
                n_unks += 1
        if (n_unks / n) > threshold:
            index_uneligible[i] = 1
    return index_uneligible.astype(bool)

array_uneligible_to = argunks(array_to, vocab)
print(array_uneligible_to.size)
array_uneligible_from = argunks(array_from, vocab)
print(array_uneligible_from.size)

3767234
3767234


In [23]:
sum(array_uneligible_to)

176344

In [24]:
sum(array_uneligible_from)

158109

In [22]:
argunks(array_to[:10], vocab)

array([False, False, False, False, False, False, False, False, False,
       False])

In [29]:
array_uneligible = array_uneligible_from | array_uneligible_to

In [30]:
sum(array_uneligible)

284861

In [32]:
array_to[array_uneligible_to][:10]

array([list(['aplaude', 'para', 'indicarle', 'que', 'ya', 'su', 'discurso', '.']),
       list(['yule', '?', 'as', 'in', 'kkkristmas', '?', '?', 'newlinechar', 'fundiealert', 'newlinechar']),
       list(['gt', 'an', 'englishman']),
       list(['aidens', '.', 'aidens', 'everywhere', '.']),
       list(['na', ',', 'elijah', 'radcliffe', '.']),
       list(['whiteisright']),
       list(['psst', '...', '.wrong', 'place', '.']),
       list(['fuckingtruth']), list(['warum', '?']),
       list(['me', 'toooo', 'd8'])], dtype=object)

In [35]:
array_uneligible_to[:100].nonzero()

(array([10, 32, 68]),)

In [36]:
array_uneligible_from[:100].nonzero()

(array([10, 46, 55, 58, 70, 92, 94]),)

In [39]:
for fname, obj in zip(['tokenized_from_clean', 'tokenized_to_clean'], 
                      [array_from[~array_uneligible], array_to[~array_uneligible]]):
    path = os.path.join(DATA_DIR, f"{fname}.pickle")
    pickle.dump(obj, open(path, "wb"), protocol=2)